# Police Force Responses by Crime

In [1]:
#Dependencies
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd

In [2]:
#Import csv file
police_data = pd.read_csv("Resources/Police_Use_of_Force.csv")
police_data.head()

,X,Y,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,Is911Call,PrimaryOffense,SubjectInjury,ForceReportNumber,...,TotalCityCallsForYear,TotalPrecinctCallsForYear,TotalNeighborhoodCallsForYear,CenterGBSID,CenterLatitude,CenterLongitude,CenterX,CenterY,DateAdded,OBJECTID
0,-93.299256,44.980557,19194596,08-007748,2008/01/08 14:20:21+00,Disturbance,No,DISCON,NaN,1,...,322402,80434.0,2514,15824,44.980557,-93.299256,-10386025.66,5618461.049,2021/02/11 08:18:54+00,1
1,-93.273109,44.979187,19194597,08-019047,2008/01/19 23:47:58+00,On Site,No,NOSMOK,NaN,1,...,322402,46998.0,23458,17239,44.979187,-93.273109,-10383115.01,5618245.449,2021/02/11 08:18:54+00,2
2,-93.311817,45.006028,19194598,08-034409,2008/02/04 21:21:49+00,Property Damage Accident,Yes,ASLT2,NaN,1,...,322402,80434.0,9486,10909,45.006028,-93.311817,-10387423.95,5622470.553,2021/02/11 08:18:54+00,3
3,-93.288860,45.043001,19194599,08-043573,2008/02/14 22:03:15+00,Unwanted Person,Yes,DAMPRP,NaN,2,...,322402,80434.0,3070,17338,45.043001,-93.288860,-10384868.44,5628293.683,2021/02/11 08:18:54+00,4
4,-93.271963,44.981655,19194600,08-052540,2008/02/24 02:25:39+00,Fight,No,OBSTRU,No,3,...,322402,46998.0,23458,17120,44.981655,-93.271963,-10382987.47,5618633.899,2021/02/11 08:18:54+00,5


In [3]:
#create new DataFrame
police_data_df = police_data.loc[:, ["PoliceUseOfForceID", "CaseNumber", "ResponseDate", "Problem", "ForceType",
                                    "ForceTypeAction", "EventAge", "TypeOfResistance", "Precinct", "Neighborhood"]]
police_data_df

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,ForceType,ForceTypeAction,EventAge,TypeOfResistance,Precinct,Neighborhood
0,19194596,08-007748,2008/01/08 14:20:21+00,Disturbance,Bodily Force,Push Away,13.0,Tensed,4,Harrison
1,19194597,08-019047,2008/01/19 23:47:58+00,On Site,Bodily Force,Push Away,27.0,Tensed,1,Downtown West
2,19194598,08-034409,2008/02/04 21:21:49+00,Property Damage Accident,Bodily Force,Joint Lock,17.0,Verbal Non-Compliance,4,Willard - Hay
3,19194599,08-043573,2008/02/14 22:03:15+00,Unwanted Person,Bodily Force,Body Weight to Pin,18.0,Tensed,4,Lind - Bohanon
4,19194600,08-052540,2008/02/24 02:25:39+00,Fight,Bodily Force,Knees,28.0,Commission of Crime,1,Downtown West
...,...,...,...,...,...,...,...,...,...,...
32030,19226626,09-Cooper,2009/01/01 00:00:00+00,NaN,NaN,0,NaN,NaN,NaN,Cooper
32031,19226627,08-Bottineau,2008/01/01 00:00:00+00,NaN,NaN,0,NaN,NaN,NaN,Bottineau
32032,19226628,09-Loring Park,2009/01/01 00:00:00+00,NaN,NaN,0,NaN,NaN,NaN,Loring Park
32033,19226629,14-Cooper,2014/01/01 00:00:00+00,NaN,NaN,0,NaN,NaN,NaN,Cooper


In [4]:
#drop NaN from Dataframe
police_data_df = police_data_df.dropna(how='any')
police_data_df

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,ForceType,ForceTypeAction,EventAge,TypeOfResistance,Precinct,Neighborhood
0,19194596,08-007748,2008/01/08 14:20:21+00,Disturbance,Bodily Force,Push Away,13.0,Tensed,4,Harrison
1,19194597,08-019047,2008/01/19 23:47:58+00,On Site,Bodily Force,Push Away,27.0,Tensed,1,Downtown West
2,19194598,08-034409,2008/02/04 21:21:49+00,Property Damage Accident,Bodily Force,Joint Lock,17.0,Verbal Non-Compliance,4,Willard - Hay
3,19194599,08-043573,2008/02/14 22:03:15+00,Unwanted Person,Bodily Force,Body Weight to Pin,18.0,Tensed,4,Lind - Bohanon
4,19194600,08-052540,2008/02/24 02:25:39+00,Fight,Bodily Force,Knees,28.0,Commission of Crime,1,Downtown West
...,...,...,...,...,...,...,...,...,...,...
30829,19225425,20-287806,2020/11/12 14:41:54+00,Domestic Abuse-In Progress,Maximal Restraint Technique,Not Side Recovery Position,19.0,Other ...,2,Waite Park
30830,19225426,20-288772,2020/11/13 21:48:02+00,Unwanted Person,Bodily Force,Body Weight Pin,29.0,Verbal Non-Compliance ...,5,Linden Hills
30831,19225427,20-292656,2020/11/19 15:44:41+00,Emotionally Disturb Person,Taser,Firing Darts,23.0,Other ...,4,Willard - Hay
30832,19225428,20-298833,2020/11/28 08:18:10+00,Motor Vehicle Chase,Bodily Force,Push Away,31.0,Fled in a Vehicle ...,3,East Phillips


In [5]:
#rename rows in ForceTypeAction column
police_data_df['ForceTypeAction'] = police_data_df['ForceTypeAction'].replace({
    "Knee": "Knees",
    "Body Weight Pin": "Body Weight to Pin",
    "Conscious Neck Restraint": "Neck Restraint: subject did not lose conciousness",
    "Punch": "Punches",
    "Slap": "Slaps",
    'Unconscious Neck Restraint': 'Neck Restraint: subject did not lose conciousness',
    'Firearm as striking tool': 'Firearms As Striking Tool'
})


/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
#rename rows in ForceType column
police_data_df['ForceType'] = police_data_df['ForceType'].replace({
    "Less Lethal Projectile": "Less Lethal"
})

/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
#rename rows in TypeOfResistance column
police_data_df['TypeOfResistance'] = police_data_df['TypeOfResistance'].replace({
    "TENSED": "Tensed",
    "tensed": "Tensed",
    "Tensed                                                                         ": "Tensed",
    "verbal non-compliance": "Verbal Non-Compliance",
    "Verbal Non-Compliance                                                          ": "Verbal Non-Compliance",
    "commission of crime":"Commission of Crime",
    "Commission of a Crime                                                          ":"Commission of Crime",
    "COMMISSION OF CRIME":"Commission of Crime",
    "Fled on Foot                                                                   ":"Fled on Foot",
    "Fled in a Vehicle                                                              ":"Fled in Vehicle",
    "Assaulted Officer                                                              ":"Assaulted Officer",
    "Assaulted Police K9                                                            ":"Assaulting Police K9",
    "Other                                                                          ":"Other"
           
})

/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
#rename columns in Dataframe
clean_df = police_data_df.rename(columns ={
    'PoliceUseOfForceID': "Police ID",
    'CaseNumber': "Case Number",
    'ResponseDate': "Response Date",
    'Problem': "Type of Crime",
    'ForceType': 'Type of Force',
    'ForceTypeAction': 'Type of Force Action',
    'EventAge': 'Event Age',
    'TypeOfResistance': 'Type of Resistance'
})
#reset the index in DataFrame
clean_df.reset_index(drop=True, inplace=True)
clean_df

,Police ID,Case Number,Response Date,Type of Crime,Type of Force,Type of Force Action,Event Age,Type of Resistance,Precinct,Neighborhood
0,19194596,08-007748,2008/01/08 14:20:21+00,Disturbance,Bodily Force,Push Away,13.0,Tensed,4,Harrison
1,19194597,08-019047,2008/01/19 23:47:58+00,On Site,Bodily Force,Push Away,27.0,Tensed,1,Downtown West
2,19194598,08-034409,2008/02/04 21:21:49+00,Property Damage Accident,Bodily Force,Joint Lock,17.0,Verbal Non-Compliance,4,Willard - Hay
3,19194599,08-043573,2008/02/14 22:03:15+00,Unwanted Person,Bodily Force,Body Weight to Pin,18.0,Tensed,4,Lind - Bohanon
4,19194600,08-052540,2008/02/24 02:25:39+00,Fight,Bodily Force,Knees,28.0,Commission of Crime,1,Downtown West
...,...,...,...,...,...,...,...,...,...,...
29070,19225425,20-287806,2020/11/12 14:41:54+00,Domestic Abuse-In Progress,Maximal Restraint Technique,Not Side Recovery Position,19.0,Other,2,Waite Park
29071,19225426,20-288772,2020/11/13 21:48:02+00,Unwanted Person,Bodily Force,Body Weight to Pin,29.0,Verbal Non-Compliance,5,Linden Hills
29072,19225427,20-292656,2020/11/19 15:44:41+00,Emotionally Disturb Person,Taser,Firing Darts,23.0,Other,4,Willard - Hay
29073,19225428,20-298833,2020/11/28 08:18:10+00,Motor Vehicle Chase,Bodily Force,Push Away,31.0,Fled in Vehicle,3,East Phillips


# 1. What incidents result the most in police use of force? (Sam)

# 2. What type of force is used the most? (Ainash)

In [ ]:
# find the most used force type
force_type_data_ainash = clean_df[['Type of Force', 'Case Number']]
force_type_data_ainash = force_type_data_ainash.groupby('Type of Force')
force_type = force_type_data_ainash['Type of Force'].count()
force_type_data = pd.DataFrame({'Total Count': force_type})
sorted_force = force_type_data.sort_values(['Total Count'], ascending = False)



In [ ]:
# create a bar chart to show the most used force type
force_bar = sorted_force.plot(kind = 'bar', title = 'Used Type of Force by Police in Minneapolis, MN area')
force_bar.set_xlabel ('Type of Force')
force_bar.set_ylabel ('Number of Cases')
plt.show()
plt.tight_layout()

In [ ]:
# create a bar chart to show the Top 5 most used force type
force_bar = sorted_force.head().plot(kind = 'bar', title = 'The Top 5 most used Type of Force by Police in Minneapolis, MN area')
force_bar.set_xlabel ('Type of Force')
force_bar.set_ylabel ('Number of Cases')
plt.show()
plt.tight_layout()

In [ ]:
# find the most used force type action
force_type_action_data_ainash = clean_df[['Type of Force Action', 'Case Number']]
force_type_action_data_ainash = force_type_action_data_ainash.groupby('Type of Force Action')
force_type = force_type_action_data_ainash['Type of Force Action'].count()
force_type_action_data = pd.DataFrame({'Total Count': force_type})
sorted_force = force_type_action_data.sort_values(['Total Count'], ascending = False)

In [ ]:
# create a bar chart to show the most used force type
force_bar = sorted_force.plot(kind = 'bar', title = 'Used Type of Force Action by Police in Minneapolis, MN area')
force_bar.set_xlabel ('Type of Force Action')
force_bar.set_ylabel ('Number of Cases')
plt.show()
plt.tight_layout()

In [ ]:
# create a bar chart to show the Top 5 most used force type
force_bar = sorted_force.head().plot(kind = 'bar', title = 'The Top 5 most used Type of Force Action by Police in Minneapolis, MN area')
force_bar.set_xlabel ('Type of Force Action')
force_bar.set_ylabel ('Number of Cases')
plt.show()
plt.tight_layout()

In [ ]:
# find the most used force type
merged_force_type_data_ainash = clean_df[['Type of Force', 'Type of Force Action', 'Case Number']]
merged_force_type_data_ainash = merged_force_type_data_ainash.groupby(['Type of Force', 'Type of Force Action'])
merged_force_type = merged_force_type_data_ainash['Type of Force'].count()
merged_force_type


# 3. Which is the most popular type of resistance? (Humera)

# 4. Which neighborhood in Minneapolis see’s the highest frequency of police force? (Mike Hills)

# 5. Is the use of police force in Minneapolis going up or down? (Mike F)

# 6. Perform a linear regression on use of force between 2015 and 2019 and make a prediction about use of force in 2020 for the city of Minneapolis.  Compare this prediction to actual use of force outcomes in 2020.